# 대화형 RAG 챗봇 구현

이 노트북은 지금까지 구현한 모든 구성 요소를 통합하여 대화형 RAG 챗봇을 구현합니다.

In [ ]:
import os
import yaml
import json
from dotenv import load_dotenv
from libs.common_utils import OpenSearch_Manager
from IPython.display import display, clear_output

load_dotenv()

## 1. 환경 설정

In [ ]:
# 모델 설정 로드
def load_model_config():
    with open("libs/config.yml", "r") as file:
        return yaml.safe_load(file)

model_config = load_model_config()

# 기본 설정
DEFAULT_TOP_K = 5
DEFAULT_TEMPERATURE = 0.7
DEFAULT_TOP_P = 0.95

# OpenSearch 매니저 초기화
os_manager = OpenSearch_Manager()
print(f"사용 가능한 인덱스: {os_manager.index_list}")

## 2. 챗봇 기능 구현

In [ ]:
def retrieve_search_results(question, bedrock_client, embed_model_id, index_name):
    """검색 결과 검색 함수"""
    response = bedrock_client.invoke_model(
        modelId=embed_model_id,
        body=json.dumps({"inputText": question})
    )
    embedding = json.loads(response['body'].read())['embedding']

    # Rank Fusion 검색 실행
    search_result = os_manager.search_by_rank_fusion(
        query_text=question,
        vector=embedding,
        index_name=index_name,
        initial_search_results=160,
        hybrid_score_filter=40,
        final_reranked_results=DEFAULT_TOP_K,
        knn_weight=0.6
    )
    
    return search_result

def format_message(role, content):
    """메시지 포맷팅 함수"""
    return {"role": role, "content": [{"text": content}]}

def chat_with_rag(question, history, bedrock_client, model_id, model_kwargs, embed_model_id, index_name):
    """RAG 기반 챗봇 대화 함수"""
    # 검색 실행
    search_result = retrieve_search_results(question, bedrock_client, embed_model_id, index_name)
    
    # 컨텍스트 준비
    context = "\n\n".join([result['content'] for result in search_result])
    
    # 시스템 프롬프트 설정
    system_prompt = [
        {"text": """당신은 도움이 되는 AI 어시스턴트입니다. 
        주어진 컨텍스트만을 기반으로 정확하고 관련성 있는 답변을 제공하세요. 
        컨텍스트에 없는 정보는 사용하지 마세요. 
        컨텍스트에서 정보를 찾을 수 없는 경우, 
        답변할 수 있는 충분한 정보가 없다고 말씀해 주세요."""}
    ]
    
    # 대화 히스토리 포맷팅
    messages = [format_message(msg["role"], msg["content"]) for msg in history]
    
    # 현재 질문에 컨텍스트 추가
    current_question = f"컨텍스트:\n{context}\n\n질문: {question}"
    messages.append(format_message("user", current_question))
    
    # Bedrock 모델 호출
    response = bedrock_client.converse(
        modelId=model_id,
        messages=messages,
        system=system_prompt,
        inferenceConfig={
            "temperature": model_kwargs["temperature"],
            "topP": model_kwargs["top_p"],
            "maxTokens": model_kwargs["max_tokens"]
        }
    )
    
    return response['output']['message']['content'][0]['text'], search_result

## 3. 대화형 인터페이스

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import boto3

# Bedrock 클라이언트 설정
bedrock_client = boto3.client('bedrock-runtime', region_name='us-west-2')
model_name = list(model_config['models'].keys())[0]
model_id = model_config['models'][model_name]['model_id']
embed_model = list(model_config['embedding_models'].keys())[0]
embed_model_id = model_config['embedding_models'][embed_model]['model_id']

# 모델 설정
model_kwargs = {
    "temperature": DEFAULT_TEMPERATURE,
    "top_p": DEFAULT_TOP_P,
    "max_tokens": 4096
}

# 대화 기록
conversation_history = []

# UI 구성요소
question_input = widgets.Text(description='질문:', layout=widgets.Layout(width='80%'))
send_button = widgets.Button(description='전송')
output = widgets.Output()
clear_button = widgets.Button(description='대화 초기화')

def on_send_button_clicked(b):
    question = question_input.value
    if question.strip():
        with output:
            print(f"\n사용자: {question}")
            response, search_results = chat_with_rag(
                question,
                conversation_history,
                bedrock_client,
                model_id,
                model_kwargs,
                embed_model_id,
                f"aws_{os_manager.index_list[0]}"
            )
            print(f"\n어시스턴트: {response}")
            print("\n참고한 문서:")
            for i, result in enumerate(search_results[:3], 1):
                print(f"\n{i}. 관련도: {result['score']:.4f}")
                print(f"내용: {result['content'][:200]}...")
        
        conversation_history.extend([
            {"role": "user", "content": question},
            {"role": "assistant", "content": response}
        ])
        question_input.value = ''

def on_clear_button_clicked(b):
    conversation_history.clear()
    with output:
        clear_output()
        print("대화가 초기화되었습니다.")

send_button.on_click(on_send_button_clicked)
clear_button.on_click(on_clear_button_clicked)

# UI 표시
display(widgets.VBox([
    widgets.HBox([question_input, send_button, clear_button]),
    output
]))